In [3]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

In [4]:
image_gen = ImageDataGenerator(ImageDataGenerator())

In [5]:
train_set = image_gen.flow_from_directory('/kaggle/input/emotion-detection-fer/train',
                                                 target_size = (48,48),
                                                 batch_size = 128,
                                                 color_mode = "grayscale",
                                                 class_mode = 'categorical')
test_set = image_gen.flow_from_directory('/kaggle/input/emotion-detection-fer/test',
                                                 target_size = (48,48),
                                                 batch_size = 128,
                                                 color_mode = "grayscale",
                                                 class_mode = 'categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
train_set.class_indices

{'angry': 0,
 'disgusted': 1,
 'fearful': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprised': 6}

In [7]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.3),
    
    keras.layers.Dense(256,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.3),
    
    keras.layers.Dense(128,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.3),
    
    keras.layers.Dense(64,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.3),
    
    keras.layers.Dense(32,activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.3),
    
    keras.layers.Dense(7,activation='softmax')

])

In [8]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_emotion_detection.h5', 
                                                monitor='accuracy', verbose=1, 
                                                mode='max',save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                         mode="max",
                                         restore_best_weights=True,
                                         patience=10)
callbacks_list = [checkpoint,early]

history = model.fit(train_set,
                     validation_data=test_set,
                     epochs=100,
                     callbacks=callbacks_list)

/opt/conda/lib/python3.7/site-packages/keras/preprocessing/image.py:1862: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  "This ImageDataGenerator specifies "


Epoch 1/100
225/225 [==============================] - ETA: 0s - loss: 2.1904 - accuracy: 0.1905
Epoch 1: accuracy improved from -inf to 0.19053, saving model to model_emotion_detection.h5
225/225 [==============================] - 338s 1s/step - loss: 2.1904 - accuracy: 0.1905 - val_loss: 1.7888 - val_accuracy: 0.2817
Epoch 2/100
225/225 [==============================] - ETA: 0s - loss: 1.8317 - accuracy: 0.2663
Epoch 2: accuracy improved from 0.19053 to 0.26629, saving model to model_emotion_detection.h5
225/225 [==============================] - 273s 1s/step - loss: 1.8317 - accuracy: 0.2663 - val_loss: 1.7277 - val_accuracy: 0.2941
Epoch 3/100
225/225 [==============================] - ETA: 0s - loss: 1.6711 - accuracy: 0.3360
Epoch 3: accuracy improved from 0.26629 to 0.33599, saving model to model_emotion_detection.h5
225/225 [==============================] - 273s 1s/step - loss: 1.6711 - accuracy: 0.3360 - val_loss: 1.6500 - val_accuracy: 0.3706
Epoch 4/100
225/225 [==========

KeyboardInterrupt: 

In [10]:
model.evaluate(test_set)

57/57 [==============================] - 12s 205ms/step - loss: 1.1481 - accuracy: 0.6023


[1.148112177848816, 0.6022568941116333]

In [ ]:
def getlabel(x):
    for i in train_set.class_indices:
        if train_set.class_indices[i] == x:
            return i

In [ ]:
predict_image("/kaggle/input/images/image.jpg")

In [ ]:
path = "/kaggle/input/emotion-detection-fer/test/angry/im0.png"
img = image.load_img(path)
img = np.expand_dims(img, axis=0)
model.predict(img)

In [ ]:
labels = [i: i in train_set.class_indices]

In [ ]:
def predict_image(path):
    img = image.load_img(path)
    img = np.expand_dims(img, axis=0)
    p = model.predict(img)
    pred = [np.argmax(element) for element in p]
    print (plt.imshow(cv2.imread(path)))
    return labels[pred[0]]